In [1]:
import torch
import torch.nn as nn  # (i.e. "neural network")
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import wandb
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from torch.utils.data import DataLoader

import tensorflow_hub as hub
USE = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/tmp/ipykernel_10651/4177980191.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-01 15:36:30.054338: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 15:36:30.280011: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 15:36

In [2]:
JSON_FILE = "./downloads/full_format_recipes.json"
dataset = pd.read_json(JSON_FILE)
dataset.head()

,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
0,"[1. Place the stock, lentils, celery, carrot, ...",7.0,2006-09-01 04:00:00+00:00,"[Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...",426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap","[4 cups low-sodium vegetable or chicken stock,...",559.0
1,[Combine first 9 ingredients in heavy medium s...,23.0,2004-08-20 04:00:00+00:00,"[Food Processor, Onion, Pork, Bake, Bastille D...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"[1 1/2 cups whipping cream, 2 medium onions, c...",1439.0
2,[In a large heavy saucepan cook diced fennel a...,7.0,2004-08-20 04:00:00+00:00,"[Soup/Stew, Dairy, Potato, Vegetable, Fennel, ...",165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"[1 fennel bulb (sometimes called anise), stalk...",165.0
3,[Heat oil in heavy large skillet over medium-h...,NaN,2009-03-27 04:00:00+00:00,"[Fish, Olive, Tomato, Sauté, Low Fat, Low Cal,...",NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,"[2 tablespoons extra-virgin olive oil, 1 cup c...",NaN
4,[Preheat oven to 350°F. Lightly grease 8x8x2-i...,32.0,2004-08-20 04:00:00+00:00,"[Cheese, Dairy, Pasta, Vegetable, Side, Bake, ...",547.0,None,20.0,3.125,Spinach Noodle Casserole,"[1 12-ounce package frozen spinach soufflé, th...",452.0


### clean the data

In [3]:
dataset = dataset[dataset['directions'].notna()]
dataset = dataset[dataset['rating'].notna()]
print("missing ingredients", dataset['ingredients'].isnull().sum())
print("missing ratings", dataset['rating'].isna().sum())
print("missing directions", dataset['directions'].isnull().sum())

missing ingredients 0
missing ratings 0
missing directions 0


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class DataFrameDataset(torch.utils.data.Dataset):
    def __init__(self, input):
        self.dataframe = pd.DataFrame()
        self.dataframe['recipe'] = input.apply(self.format, axis=1)

        # soup = self.dataframe['categories'].apply(lambda x: 'Soup/Stew' in str(x))
        # sandwich = self.dataframe['categories'].apply(lambda x: 'Sandwich' in str(x))
        # salad = self.dataframe['categories'].apply(lambda x: 'Salad' in str(x))

        def categorize_row(x):
            if 'Soup/Stew' in str(x):
                return [1,0,0,0]
            elif 'Sandwich' in str(x):
                return [0,1,0,0]
            elif 'Salad' in str(x):
                return [0,0,1,0]
            else:
                return [0,0,0,1]

        self.dataframe['category'] = input['categories'].apply(categorize_row)


        recipe = self.dataframe['recipe']
        good = recipe.apply(lambda x: isinstance(x, torch.Tensor))
        self.dataframe = self.dataframe[good]
        
        # non_tensors = self.dataframe['recipe'].map(lambda x: not isinstance(x, torch.Tensor))
        # self.dataframe['recipe'] = self.dataframe['recipe'][non_tensors]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input = self.dataframe.iloc[idx]['recipe']
        label = self.dataframe.iloc[idx]['category']
        return torch.tensor(input, dtype=torch.float), torch.tensor(label, dtype=torch.float)

    def embed(self, sentence):
        embedding = USE([sentence])
        return  torch.from_numpy(embedding.numpy()).float().squeeze(0)

    def format(self, data):
        ingredients = " ".join(data['ingredients']) if data['ingredients'] else ""
        steps = " ".join(data['directions']) if data['directions'] else ""
        recipe = f"TITLE: {data['title']} \nDESCRIPTION: {data['desc']} \nINGREDIENTS: {ingredients} \nINSTRUCTIONS: {steps}"
        return self.embed(recipe)
    
    


In [38]:
clean_data = DataFrameDataset(dataset)



In [40]:
# Convert one-hot encoded lists to categorical labels
clean_data.dataframe['categorical_label'] = clean_data.dataframe['category'].apply(lambda x: x.index(1) if 1 in x else -1)

# Count occurrences of each categorical label
category_counts = clean_data.dataframe['categorical_label'].value_counts()

print(category_counts)

categorical_label
3    16859
2     1649
0     1164
1      428
Name: count, dtype: int64


In [41]:
# 70% of processed_dataset is used for training
train_size = int(0.76 * len(clean_data))
# 15% of clean_data is used for validation
val_size = int(0.12 * len(clean_data))
# The remaining 15% is used for testing
test_size = len(clean_data) - train_size - val_size


train_data, val_data, test_data = torch.utils.data.random_split(
   clean_data, [train_size, val_size, test_size]
)

print(f"Total size: {len(clean_data)}")
print(f"Train size: {len(train_data)}")
print(f"Test size: {len(test_data)}")
print(f"Val size: {len(val_data)}")



Total size: 20100
Train size: 15276
Test size: 2412
Val size: 2412


In [21]:
def get_category_counts(data_subset, original_dataframe):

    # Extract indices of the subset
    subset_indices = data_subset.indices
    # Filter the original dataframe using these indices
    filtered_df = original_dataframe.iloc[subset_indices]
    # Count occurrences of each category
    category_counts = filtered_df['category'].value_counts()
    return category_counts

# Assuming `clean_data.dataframe` is your original dataframe and contains the 'category' column
print("TRAIN Category Counts:\n", get_category_counts(train_data, clean_data.dataframe))
print("TEST Category Counts:\n", get_category_counts(test_data, clean_data.dataframe))
print("VALIDATE Category Counts:\n", get_category_counts(val_data, clean_data.dataframe))

TRAIN Category Counts:
 category
[0, 0, 0, 1]    12800
[0, 0, 1, 0]     1273
[1, 0, 0, 0]      870
[0, 1, 0, 0]      333
Name: count, dtype: int64
TEST Category Counts:
 category
[0, 0, 0, 1]    2036
[0, 0, 1, 0]     195
[1, 0, 0, 0]     136
[0, 1, 0, 0]      45
Name: count, dtype: int64
VALIDATE Category Counts:
 category
[0, 0, 0, 1]    2023
[0, 0, 1, 0]     181
[1, 0, 0, 0]     158
[0, 1, 0, 0]      50
Name: count, dtype: int64


In [42]:

class SSSModel(nn.Module):
    def __init__(self):
        super(SSSModel, self).__init__()
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 4)

    def forward(self, x):
        # Pass data through fc1
        x = self.fc1(x)
        # Use ReLU activation function
        x = F.relu(x)
        # Pass data through fc2
        x = self.fc2(x)
        # UsF.ReLU activation function
        x = F.relu(x)
        # Pass data through fc3
        x = self.fc3(x)
        # x = F.softmax(x, dim=1)
        return x



In [ ]:
print(type(train_data))

BATCH_SIZE = 32
# Number of epochs to train for
epochs = 3



model = SSSModel()
model.to(device)
model.train()


# Create a DataLoader with batch size of 32
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

# Define an optimizer - Stochastic Gradient Descent is used here as an example
optimizer = optim.SGD(model.parameters(), lr=0.0001)

loss_history = []


<class 'torch.utils.data.dataset.Subset'>


In [47]:
# Train the model
print(epochs)
for epoch in range(epochs):
    total_lost = 0

    for inputs, targets in train_loader:

        # print(inputs.shape, targets.shape)
        # print(targets[0].shape)

        # print(inputs, targets)
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs.to(device))

        # Calculate loss
        # print("Pred min, max: ", outputs.shape, outputs.min().item(), outputs.max().item())
        # print("Label min, max: ", targets.shape, targets.min().item(), targets.max().item())
        loss = criterion(outputs, torch.max(targets, 1)[1])

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        total_lost += loss.item()

    loss_history.append(total_lost/ len(train_loader))
    print(f"Epoch {epoch+1}/{epochs} Loss: {loss.item()}")


# Plotting the loss history
plt.plot(loss_history)
plt.title('Loss History')
plt.xlabel('Batch Number')
plt.ylabel('Loss')
plt.show()

3
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384950>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384950>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor obje

/tmp/ipykernel_10651/2750850707.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(input, dtype=torch.float), torch.tensor([label], dtype=torch.float)


<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384950>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384950>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384890>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object at 0x7f84c8384950>
<built-in method values of Tensor object at 0x7f84c8384a10>
<built-in method values of Tensor object

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [108]:
def test_train_loader(train_loader):
    for i, (data, labels) in enumerate(train_loader):
        # Check if data and labels are of type torch.Tensor
        assert isinstance(data, torch.Tensor), f"Data in batch {i} is not a torch.Tensor"
        assert isinstance(labels, torch.Tensor), f"Labels in batch {i} is not a torch.Tensor"
        
        # Check the dimensions of the data and labels
        # Assuming data should have dimensions [batch_size, 512] and labels [batch_size, 1]
        assert data.dim() == 2, f"Data in batch {i} does not have 2 dimensions"
        assert data.shape[1] == 512, f"Data in batch {i} does not have 512 features, has {data.shape[1]}"
        assert labels.dim() == 2, f"Labels in batch {i} do not have 2 dimensions"
        assert labels.shape[1] == 1, f"Labels in batch {i} are not single values, shape is {labels.shape}"
        
        # Optionally, check the range of data and labels if applicable
        # For example, if labels are ratings between 0 and 5
        assert labels.min() >= 0, f"Labels in batch {i} contain values less than 0"
        assert labels.max() <= 1, f"Labels in batch {i} contain values greater than 5"
        
        print(f"Batch {i} passed all tests.")

# Assuming train_loader is your DataLoader instance
test_train_loader(train_loader)


Batch 0 passed all tests.
Batch 1 passed all tests.


/tmp/ipykernel_3228/2007158972.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(input, dtype=torch.float), torch.tensor([label], dtype=torch.float)


RuntimeError: stack expects each tensor to be equal size, but got [512] at entry 0 and [] at entry 1